using the by Hour data

In [1]:
import pandas as pd, matplotlib as plt , seaborn as sns, numpy as np, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('content/preprocessed_train.csv').query('building_id==122 & meter==0').drop(columns= ['Unnamed: 0','precip_depth_1_hr','cloud_coverage', 'site_id', 'square_feet', 'building_id','primary_use', 'meter'] )

In [3]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index('timestamp', inplace=True)

In [4]:
df.head()

,meter_reading,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
timestamp,,,,,,,,,
2016-01-01 00:00:00,134.3,3.8,2.4,1020.9,240.0,3.1,1,1,0
2016-01-01 01:00:00,267.1,3.7,2.4,1021.6,230.0,2.6,1,1,1
2016-01-01 02:00:00,268.7,2.6,1.9,1021.9,0.0,0.0,1,1,2
2016-01-01 03:00:00,267.7,2.0,1.2,1022.3,170.0,1.5,1,1,3
2016-01-01 04:00:00,267.8,2.3,1.8,1022.7,110.0,1.5,1,1,4


In [62]:
scaler=MinMaxScaler()
# only scaling the numerical values not day/month
scaled_data=scaler.fit_transform(df.drop(columns=['day','month','hour']))

scaled_df= pd.concat( [ pd.DataFrame(scaled_data, index=df.index , columns=df.columns[:-3]) , df.iloc[:,-3:]  ] 
                     , axis=1 )
scaled_df.head()

,meter_reading,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
timestamp,,,,,,,,,
2016-01-01 00:00:00,0.000000,0.209239,0.381757,0.659280,0.666667,0.167568,1,1,0
2016-01-01 01:00:00,0.364635,0.206522,0.381757,0.668975,0.638889,0.140541,1,1,1
2016-01-01 02:00:00,0.369028,0.176630,0.364865,0.673130,0.000000,0.000000,1,1,2
2016-01-01 03:00:00,0.366282,0.160326,0.341216,0.678670,0.472222,0.081081,1,1,3
2016-01-01 04:00:00,0.366557,0.168478,0.361486,0.684211,0.305556,0.081081,1,1,4


In [116]:
filled_data= scaled_df.fillna(method='ffill', axis=1 )

In [117]:
x_train,x_val,y_train,y_val= train_test_split( filled_data.drop(columns='meter_reading') ,
                                                    filled_data.meter_reading,
                                                    test_size=0.2,
                                                    shuffle=False )
assert len(x_train) == len(y_train)

In [118]:
y_val

timestamp
2016-10-19 20:00:00    0.335255
2016-10-19 21:00:00    0.286107
2016-10-19 22:00:00    0.276496
2016-10-19 23:00:00    0.266337
2016-10-20 00:00:00    0.266612
                         ...   
2016-12-31 19:00:00    0.292696
2016-12-31 20:00:00    0.285283
2016-12-31 21:00:00    0.280066
2016-12-31 22:00:00    0.275947
2016-12-31 23:00:00    0.278418
Name: meter_reading, Length: 1753, dtype: float64

In [119]:
x_val

,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
timestamp,,,,,,,,
2016-10-19 20:00:00,0.413043,0.520270,0.668975,0.888889,0.140541,19.0,10.0,20.0
2016-10-19 21:00:00,0.402174,0.530405,0.670360,0.861111,0.140541,19.0,10.0,21.0
2016-10-19 22:00:00,0.391304,0.523649,0.671745,0.833333,0.167568,19.0,10.0,22.0
2016-10-19 23:00:00,0.366848,0.523649,0.670360,0.861111,0.113514,19.0,10.0,23.0
2016-10-20 00:00:00,0.334239,0.513514,0.667590,0.888889,0.113514,20.0,10.0,0.0
...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,0.326087,0.520270,0.750693,0.611111,0.194595,31.0,12.0,19.0
2016-12-31 20:00:00,0.301630,0.506757,0.742382,0.611111,0.221622,31.0,12.0,20.0
2016-12-31 21:00:00,0.293478,0.496622,0.732687,0.611111,0.248649,31.0,12.0,21.0


In [120]:
# creating csv files of the data
list_of_df= [x_train , y_train,
            x_val , y_val]

list_of_df_names= ['x_train_h' , 'y_train_h',
                    'x_val_h' , 'y_val_h']

for i, frame in enumerate(list_of_df):
    frame.to_csv( f'content/{list_of_df_names[i]}.csv')

In [121]:
list_of_df=  ['x_train_h' , 'y_train_h',
                    'x_val_h' , 'y_val_h']

for i, frame in enumerate(list_of_df):
    list_of_df[i]= pd.read_csv( f'content/{list_of_df[i]}.csv')
    
x_train , y_train, x_val , y_val = list_of_df
            

In [122]:
x_train.head()

,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,day,month,hour
0,2016-01-01 00:00:00,0.209239,0.381757,0.659280,0.666667,0.167568,1.0,1.0,0.0
1,2016-01-01 01:00:00,0.206522,0.381757,0.668975,0.638889,0.140541,1.0,1.0,1.0
2,2016-01-01 02:00:00,0.176630,0.364865,0.673130,0.000000,0.000000,1.0,1.0,2.0
3,2016-01-01 03:00:00,0.160326,0.341216,0.678670,0.472222,0.081081,1.0,1.0,3.0
4,2016-01-01 04:00:00,0.168478,0.361486,0.684211,0.305556,0.081081,1.0,1.0,4.0


In [123]:
train_gen= tf.keras.preprocessing.sequence.TimeseriesGenerator(np.array( x_train.drop(columns='timestamp')), 
                                                               np.array( y_train.drop(columns='timestamp')) , 
                                                                length=6 ,sampling_rate=1, 
                                                                stride=1, batch_size=32
                                                                )

val_gen= tf.keras.preprocessing.sequence.TimeseriesGenerator(  np.array(x_val.drop(columns='timestamp') ), 
                                                               np.array(y_val.drop(columns='timestamp') ) , 
                                                                length=6 ,sampling_rate=1, 
                                                                stride=1, batch_size=32
                                                                )


In [137]:
model= tf.keras.Sequential([tf.keras.layers.LSTM(32 , activation='relu' ,return_sequences=False),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1) ] )

In [138]:
model.compile(loss='mse', optimizer= tf.keras.optimizers.Adam(0.001))

cb= tf.keras.callbacks.ModelCheckpoint('models/LSTM_per_hour_model', monitor='val_loss', verbose=0, save_best_only=True)
# Fitting the model
history=model.fit(train_gen, 
          validation_data= val_gen,
          epochs=30 ,
          callbacks=[cb],
          shuffle=False)

Epoch 1/30
214/219 [============================>.] - ETA: 0s - loss: 0.3619WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models\LSTM_per_hour_model\assets
219/219 [==============================] - 9s 41ms/step - loss: 0.3544 - val_loss: 0.0131
Epoch 2/30
219/219 [==============================] - 9s 41ms/step - loss: 0.0382 - val_loss: 0.0115
Epoch 3/30
219/219 [==============================] - 9s 43ms/step - loss: 0.0252 - val_loss: 0.0110
Epoch 4/30
219/219 [==============================] - 1s 5ms/step - loss: 0.0193 - val_